In [2]:
# !pip install torchcsprng
!pip install opacus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 4.5 MB/s 
     |██████████████████████████████  | 834.1 MB 40.6 MB/s eta 0:00:02tcmalloc: large alloc 1147494400 bytes == 0x65f5a000 @  0x7efc21833615 0x5d631c 0x51e4f1 0x51e67b 0x4f7585 0x49ca7c 0x4fdff5 0x49caa1 0x4fdff5 0x49ced5 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x5d7c18 0x5d9412 0x586636 0x5d813c 0x55f3fd 0x55e571 0x5d7cf1 0x49ced5 0x55e571 0x5d7cf1 0x49ec69 0x5d7c18 0x49ca7c 0x4fdff5 0x49ced5
     |████████████████████████████████| 890.2 MB 4.9 kB/s 
     |████████████████████████████████| 21.0 MB 1.3 MB/s 
     |████████████████████████████████| 317.1 MB 32 kB/s 
     |████████████████████████████████| 849 kB 48.0 MB/s 
     |████████████████████████████████| 557.1 MB 8.1 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uni

In [3]:
# load dataset
import torch 
from torch.utils.data import DataLoader
import torchvision.datasets as dsets 
import torchvision.transforms as transforms
random_seed = 6
torch.manual_seed(random_seed)

batch_size = 200
# MNIST dataset
train_dataset = dsets.MNIST(root='./pymnist', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = dsets.MNIST(root='./pymnist', train=False, transform=transforms.ToTensor(), download=True)
# load_data
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


/usr/local/lib/python3.8/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /usr/local/lib/python3.8/dist-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./pymnist/MNIST/raw/train-images-idx3-ubyte.gz to ./pymnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./pymnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./pymnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./pymnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./pymnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./pymnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./pymnist/MNIST/raw



In [4]:
# original_data
print("train_data:", train_dataset.train_data.size())
print("train_labels:", train_dataset.train_labels.size())
print("test_data:", test_dataset.test_data.size())
print("test_labels:", test_dataset.test_labels.size())
# shuffle batch_size data
print("batch_size:", train_loader.batch_size)
print("load_train_data:", train_loader.dataset.train_data.shape)
print("load_train_labels:", train_loader.dataset.train_labels.shape)


train_data: torch.Size([60000, 28, 28])
train_labels: torch.Size([60000])
test_data: torch.Size([10000, 28, 28])
test_labels: torch.Size([10000])
batch_size: 200
load_train_data: torch.Size([60000, 28, 28])
load_train_labels: torch.Size([60000])


/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [5]:
# target model, only has two layers
from torch import nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(3, 16, 5)
        self.fc1 = nn.Linear(256, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 256)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [13]:
#  model with noise
from opacus import PrivacyEngine
# import torchcsprng as prng
criterion = nn.CrossEntropyLoss()
privacy_engine = PrivacyEngine(
            secure_mode=None,
        )
dpnet = Net()
learning_rate = 1e-1

optimizer = torch.optim.SGD(dpnet.parameters(), lr=learning_rate)

dpnet, optimizer, train_loader = privacy_engine.make_private(
            module=dpnet,
            optimizer=optimizer,
            data_loader=train_loader,
            noise_multiplier=4, # noise multiplier
            max_grad_norm=10.0,
            clipping="flat",
        )


for epoch in range(5):
    print("current epoch = {}".format(epoch))
    for i, (images,labels) in enumerate(train_loader):
        # print(images.shape)
        # images = (images.view(-1, 28*28))
        labels = (labels)
        # print(labels)

        outputs = dpnet(images)
        # print(outputs)
        # print(labels)
        loss = criterion(outputs, labels)  # calculate loss
        optimizer.zero_grad()  # clear net state before backward
        loss.backward()       
        optimizer.step()   # update parameters

        if i%5000 == 0:
            print("current loss = %.5f" %loss.item())
print("finished training")

current epoch = 0
current loss = 2.30449
current epoch = 1
current loss = 2.23342
current epoch = 2
current loss = 5.79129
current epoch = 3
current loss = 6.30276
current epoch = 4
current loss = 11.76049
finished training


In [ ]:
torch.save(dpnet.state_dict(), "/content/withDP.pth") # save model with noise


In [7]:
# target model
net = Net()
print(net)
learning_rate = 1e-1
num_epoches = 5
# criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)
for epoch in range(num_epoches):
    print("current epoch = {}".format(epoch))
    for i, (images,labels) in enumerate(train_loader):
        # print(images.shape)
        # images = (images.view(-1, 28*28))
        labels = (labels)
        # print(labels)

        outputs = net(images)
        # print(outputs)
        # print(labels)
        loss = criterion(outputs, labels)  # calculate loss
        optimizer.zero_grad()  # clear net state before backward
        loss.backward()       
        optimizer.step()   # update parameters

        if i%5000 == 0:
            print("current loss = %.5f" %loss.item())
print("finished training")


Net(
  (conv1): Conv2d(1, 3, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
current epoch = 0
current loss = 2.30506
current epoch = 1
current loss = 0.25030
current epoch = 2
current loss = 0.10640
current epoch = 3
current loss = 0.09460
current epoch = 4
current loss = 0.11377
finished training


In [ ]:
torch.save(net.state_dict(), "/content/shadow.pth") # save model with no noise


In [ ]:
total = 0
correct = 0
for images, labels in test_loader:
    # images = (images.view(-1, 28*28))
    labels = (labels)
    outputs = net(images)

    _,predicts = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicts == labels).sum()
print("Accuracy = %.2f" %(100*correct/total))


Accuracy = 98.20


In [8]:
# thieft model, has three layers
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, 3)
        self.pool = nn.MaxPool2d(1, 1)
        self.conv2 = nn.Conv2d(3, 6, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(6400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # print(x.shape)
        x = x.view(-1, 6400)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [9]:
# test closeness 相似性
def testd(model,shadow,test_loader):
    total = 0
    correct = 0
    for images, labels in test_loader:
        # images = (images.view(-1, 28*28))
        images = images.to(torch.float32)
        labels2 = shadow(images)
        _, predicts2 = torch.max(labels2.data, 1)


        labels = (labels)
        outputs = model(images)

        _,predicts = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicts == predicts2).sum()
    print("Accuracy = %.2f" %(100*correct/total))

In [10]:
# test accuracy 准确率
def testr(model,test_loader):
  total = 0
  correct = 0
  for images, labels in test_loader:
      # images = (images.view(-1, 28*28))
      images = images.to(torch.float32)

      labels = (labels)
      outputs = model(images)

      _,predicts = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicts == labels).sum()
  print("Accuracy = %.2f" %(100*correct/total))


#根据影子模型（未知模型）训练新模型
def shadow(shadownet):
    model = Model()
    print(model)
    learning_rate2 = 1e-1
    num_epoches2 = 3
    criterion2 = nn.CrossEntropyLoss()
    optimizer2 = torch.optim.SGD(model.parameters(), lr=learning_rate2)
    # train_dataset=my_trainData_Set()
    # test_dataset=my_testData_Set()

    train_dataset = dsets.MNIST(root='./pymnist', train=True, transform=transforms.ToTensor(), download=True)
    test_dataset = dsets.MNIST(root='./pymnist', train=False, transform=transforms.ToTensor(), download=True)
    train_loader2 = DataLoader(dataset=train_dataset, batch_size=200, shuffle=True)
    test_loader2 = DataLoader(dataset=test_dataset, batch_size=200, shuffle=False)
    newLabel = []

    # shadow = Net()
    # shadow.load_state_dict(torch.load('/content/shadow.pth'))
    # shadow.eval()
    print("未知模型的准确率")
    testr(shadownet,test_loader2)
    print("新模型没训练前的准确率")

    testr(model,test_loader2)
    print("similarity:")
    testd(model,shadownet,test_loader2)
    for epoch in range(num_epoches2):
        print("current epoch = {}".format(epoch))
        for i, (images, labels) in enumerate(train_loader2):
            # print(images.shape)
            # images = (images.view(-1, 28*28))
            # labels = (labels)
            images = images.to(torch.float32)

            labels2 = shadownet(images)

            # print(labels)
            # labels3=labels2.detach().numpy()
            # newLabel.append(labels3)
            # print(predicts)

            # print(newLabel)

            outputs = model(images)
            _, predicts = torch.max(labels2.data, 1)

            # print(outputs.shape)
            # print(predicts.shape)
            # print(predicts)
            # print("**")
            # print(labels)
            # print("--")

            loss2 = criterion2(outputs, predicts)  # calculate loss)
            optimizer2.zero_grad()  # clear net state before backward
            loss2.backward()
            optimizer2.step()  # update parameters

            if i % 20 == 0:
                print("current loss = %.5f" % loss2.item())
    print("finished training 后的新模型的")
    testr(model,test_loader2)
    print("similarity:")
    testd(model,shadownet,test_loader2)
    

In [11]:
# function to test accuracy
def test(model,test_loader):
  total = 0
  correct = 0
  for images, labels in test_loader:
      # images = (images.view(-1, 28*28))
      labels = (labels)
      outputs = model(images)

      _,predicts = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicts == labels).sum()
  print("Accuracy = %.2f" %(100*correct/total))


In [12]:
shadow(net)

Model(
  (conv1): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=6400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
未知模型的准确率
Accuracy = 98.12
新模型没训练前的准确率
Accuracy = 10.32
similarity:
Accuracy = 10.58
current epoch = 0
current loss = 2.30133
current loss = 2.30392
current loss = 2.30033
current loss = 2.27655
current loss = 1.95357
current loss = 2.31072
current loss = 2.25488
current loss = 1.61428
current loss = 0.92043
current loss = 0.66932
current loss = 0.36428
current loss = 0.27146
current loss = 0.26485
current loss = 0.26880
current loss = 0.26340
current epo

In [14]:
shadow(dpnet)


Model(
  (conv1): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=6400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
未知模型的准确率
Accuracy = 49.82
新模型没训练前的准确率
Accuracy = 9.82
similarity:
Accuracy = 8.26
current epoch = 0
current loss = 2.30197
current loss = 2.24133
current loss = 2.11001
current loss = 2.05895
current loss = 1.64481
current loss = 1.57965
current loss = 1.23354
current loss = 1.18259
current loss = 1.33254
current loss = 1.34617
current loss = 1.24961
current loss = 1.22346
current loss = 1.14696
current loss = 1.13396
current loss = 1.02159
current epoch

Accuracy = 9.74
